In [ ]:
!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git
%cd Sakura-13B-Galgame
!pip install -q -U torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install -q auto-gptq
!pip install -q -r requirements.txt
!pip install -q pyngrok

# install localtunnel
!npm install -g localtunnel

In [ ]:
# ngrokToken留空则使用localtunnel进行内网穿透
ngrokToken = "2jGyaSU2qDyRYae0kRsUQ7U6PJ4_68FBmrHxsyRe2zg1HV3yU"
use_pinggy = True
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"


if ngrokToken:
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(5000)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
elif use_pinggy:
    import subprocess
    import threading
    def start_pinggy(port):
        cmd = f"ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -p 443 -R0:localhost:{port} a.pinggy.io"
        p = subprocess.Popen(cmd.split(" "), stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')
    threading.Thread(target=start_pinggy, daemon=True, args=(5000,)).start()
else:
    import subprocess
    import threading
    def start_localtunnel(port):
        p = subprocess.Popen(["lt", "--port", f"{port}"], stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')
    threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()

while True:
    !python server.py \
  --model_name_or_path $MODEL \
  --use_gptq_model \
  --model_version 0.8 \
  --trust_remote_code \
  --no-auth